In [1]:
import os
import re
import pandas as pd
from datetime import datetime

path = 'D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\\' # Dataset folder路徑

## Pulse data 貼標

In [44]:
def addLabels(init_path):
    folder_list = getFolderList(init_path)
    for folder in folder_list:
        addLabelToFiles(folder)

def getFolderList(init_path):
    p = re.compile('.git')
    folder_list = []
    for file in os.listdir(init_path):
        if p.match(file): continue #不納入.git folder
        if os.path.isdir(os.path.join(init_path,file)):
            folder_list.append(os.path.join(init_path, file))
    return folder_list
    
def addLabelToFiles(folder):
    files = os.listdir(folder)
    for file in files:
        p = re.compile('labels.csv')
        if p.match(file):
            file_path = os.path.join(folder, file)
            appendLabelAndWriteFile(file_path)

def appendLabelAndWriteFile(file_path):
    labels_df = pd.read_csv(file_path, engine='python', header=None)
    print('folder %s' %os.path.dirname(file_path).split('\\')[-1] + '......')
    for index, row in labels_df.iterrows():
        num = row[0]
        folder_path = '\\'.join(file_path.split('\\')[:-1])
        pulse_df = pd.read_csv(folder_path + '\\pulse_%s.csv' %num, engine='python')
        pulse_df['label'] = ''
        thresh_str = row[1]
        thresh_time = datetime.strptime(thresh_str, '%H:%M:%S') # thresh時間
        labelled_pulse_df = addLabelToPulse(pulse_df, thresh_time)
        pulse_df.to_csv(folder_path + '\\labelled_pulse_%s.csv' %num, index=None) # 產生 labelled_pulse.csv
        print(folder_path + '\\labelled_pulse_%s.csv' %num + ' labels added')
    print('folder %s' %os.path.dirname(file_path).split('\\')[-1] + ' finish......\n') 

def addLabelToPulse(pulse_df, thresh_time):
    label_list = []
    for index, row in pulse_df.iterrows():
        pulse_time = datetime.strptime(row['datetime'].split(' ')[1], '%H:%M:%S') # pulse時間
        if pulse_time >= thresh_time:
            label_list.append(1)
        else:
            label_list.append(0)
    pulse_df['label'] = pd.Series(label_list, index=pulse_df.index)
    return pulse_df

In [45]:
addLabels(path)

folder 2018-02-28......
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_1.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_2.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_3.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_4.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_5.csv labels added
folder 2018-02-28 finish......

folder 2018-03-01......
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_1.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_2.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_3.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_4.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_5.csv labels added
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-

## Create dof_table.csv & pulse_table.csv

In [41]:
def getFileList(path, p):
    files = []
    for i in os.listdir(path):
        folder_path = os.path.join(path,i)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if p.match(file):
                    file_path = os.path.join(folder_path, file)
                    print(file_path)
                    files.append(file_path)
    return files

def appendToSingleTable(files):
    df = pd.DataFrame()
    print('\nAppending tables......')
    for file in files:
        print(file)
        df = df.append(pd.read_csv(file, engine='python'), ignore_index=True)
    return df

In [42]:
# 取得所有dof檔案的路徑
print('Retrieving data from 9_dof files...')
p = re.compile('9_dof_\d+.csv')
files = getFileList(path, p)

df = appendToSingleTable(files)                                               # merge資料
df.to_csv(path + 'dof_table.csv', index=False, sep=',', encoding='utf-8')     # 寫出檔案
print('Merge data from 9_dof files complete!')

# 取得所有pulse檔案的路徑
print('\nRetrieving data from labelled_pulse files...')
p = re.compile('labelled_pulse_\d+.csv')
files = getFileList(path, p)

df = appendToSingleTable(files)                                               # merge資料
df.to_csv(path + 'pulse_table.csv', index=False, sep=',', encoding='utf-8')   # 寫出檔案
print('Merge data from labelled_pulse files complete!')

Retrieving data from 9_dof files...
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\9_dof_1.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\9_dof_2.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\9_dof_3.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\9_dof_4.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\9_dof_5.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_1.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_2.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_3.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_4.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_5.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_6.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_7.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\9_dof_8.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-02\9_dof_1.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-02\9

## Create join_table.csv

In [49]:
pulse_df = pd.read_csv(path + 'pulse_table.csv', engine='python')
dof_df = pd.read_csv(path + 'dof_table.csv', engine='python')

dof_df['datetime'] = pd.to_datetime(dof_df['datetime'])     # to datetime datatype
dof_df['datetime2'] = dof_df['datetime'].dt.round('1s')     # round to nearest second & create a datetime column called 'datetime2'
pulse_df['datetime'] = pd.to_datetime(pulse_df['datetime']) # to datetime datatype

joined_df = dof_df.join(pulse_df.set_index('datetime'), on='datetime2', how='inner')
del joined_df['datetime2']
joined_df.to_csv(path + 'joined_table.csv', index=False)
print(path + 'joined_table.csv..... FILE CREATED!')

D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\joined_table.csv..... FILE CREATED!
